# Track an experiment while training a Pytorch model locally or in your notebook

---
## Introductions

This notebook shows how you can use the SageMakerCore SDK to track a Machine Learning experiment using a Pytorch model trained locally.



### Experiment
An experiment is a collection of runs. When you initialize a run in your training loop, you include the name of the experiment that the run belongs to. Experiment names must be unique within your AWS account.

## Pre-Requisites

### Install Latest SageMakerCore
All SageMakerCore beta distributions will be released to a private s3 bucket. After being allowlisted, run the cells below to install the latest version of SageMakerCore from `s3://sagemaker-core-beta-artifacts/sagemaker_core-latest.tar.gz`

Ensure you are using a kernel with python version >=3.8

In [ ]:
# Uninstall previous version of sagemaker-core and restart kernel
!pip uninstall sagemaker-core -y

In [ ]:
# Make dist/ directory to hold the sagemaker-core beta distribution file
!mkdir dist

In [ ]:
# Download and Install the latest version of sagemaker-core
!aws s3 cp s3://sagemaker-core-beta-artifacts/sagemaker_core-latest.tar.gz dist/

!pip install dist/sagemaker_core-latest.tar.gz

In [ ]:
# Check the version of sagemaker-core
!pip show -v sagemaker-core

In [ ]:
### Install Additional Packages

In [ ]:
# Install additionall packages

!pip install -U torch torchvision matplotlib

### Setup

Import required libraries and set logging and experiment configuration

In [ ]:
from torchvision import datasets, transforms
import torch
import os
import time
from matplotlib import pyplot as plt
from sagemaker.core.helper.session_helper import Session
from sagemaker.core.utils import get_textual_rich_logger

logger = get_textual_rich_logger(__name__)
session = Session()
region = session.boto_region_name

experiment_name = "local-pyspark-experiment-example-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
run_group_name = "Default-Run-Group-" + experiment_name
run_name = "local-experiment-run-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

### Download the dataset

Let's now use the torchvision library to download the MNIST dataset from tensorflow and apply a transformation on each image

In [ ]:
# download the dataset
# this will not only download data to ./mnist folder, but also load and transform (normalize) them
datasets.MNIST.urls = [
    f"https://sagemaker-example-files-prod-{region}.s3.amazonaws.com/datasets/image/MNIST/train-images-idx3-ubyte.gz",
    f"https://sagemaker-example-files-prod-{region}.s3.amazonaws.com/datasets/image/MNIST/train-labels-idx1-ubyte.gz",
    f"https://sagemaker-example-files-prod-{region}.s3.amazonaws.com/datasets/image/MNIST/t10k-images-idx3-ubyte.gz",
    f"https://sagemaker-example-files-prod-{region}.s3.amazonaws.com/datasets/image/MNIST/t10k-labels-idx1-ubyte.gz",
]

train_set = datasets.MNIST(
    "mnist_data",
    train=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
    download=True,
)

test_set = datasets.MNIST(
    "mnist_data",
    train=False,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
    download=True,
)

In [ ]:
s3_client = session.s3_client
bucket_name = session.default_bucket()
for f in os.listdir(train_set.raw_folder):
    file_path = train_set.raw_folder + "/" + f
    s3_client.upload_file(file_path, bucket_name, file_path)

View and example image from the dataset

In [ ]:
plt.imshow(train_set.data[2].numpy())

### Create experiment and log dataset information

Create an experiment run to track the model training. SageMaker Experiments is a great way to organize your data science work. You can create an experiment to organize all your model runs and analyse the different model metrics with the SageMaker Experiments UI.

Here we create an experiment together with a trial and trial component for it. We also log all the downloaded files as inputs to our model.

In [ ]:
from sagemaker.core.resources import Experiment, Trial, TrialComponent
from sagemaker.core.shapes import TrialComponentParameterValue, TrialComponentArtifact

experiment = Experiment.create(experiment_name=experiment_name)
trial = Trial.create(trial_name=run_group_name, experiment_name=experiment_name)

trial_component_parameters = {
    "num_train_samples": TrialComponentParameterValue(number_value=len(train_set.data)), 
    "num_test_samples": TrialComponentParameterValue(number_value=len(test_set.data)),
}

# Setting input dataset file path
trial_component_input_artifacts = {}
for f in os.listdir(train_set.raw_folder):
    file_path = train_set.raw_folder + "/" + f
    trial_component_input_artifacts[f] = TrialComponentArtifact(value=file_path)

trial_component = TrialComponent.create(
    trial_component_name=run_name,
    parameters=trial_component_parameters,
    input_artifacts=trial_component_input_artifacts,
)
trial_component.associate_trail(trial_name=run_group_name)

Checking the SageMaker Experiments UI, you can observe that a new Experiment was created with the run associated to it.

<img src="images/experiment_created.png" width="100%" style="float: left;" />

### Create model training functions

Create an experiment run to track the model training. SageMaker Experiments is a great way to organize your data science work. You can create an experiment to organize all your model runs and analyse the different model metrics with the SageMaker Experiments UI.

Here we create an experiment run and log parameters for the size of our training and test datasets. We also log all the downloaded files as inputs to our model.

In [ ]:
# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(torch.nn.Module):
    def __init__(self, hidden_channels, kernel_size, drop_out):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, hidden_channels, kernel_size=kernel_size)
        self.conv2 = torch.nn.Conv2d(hidden_channels, 20, kernel_size=kernel_size)
        self.conv2_drop = torch.nn.Dropout2d(p=drop_out)
        self.fc1 = torch.nn.Linear(320, 50)
        self.fc2 = torch.nn.Linear(50, 10)

    def forward(self, x):
        x = torch.nn.functional.relu(torch.nn.functional.max_pool2d(self.conv1(x), 2))
        x = torch.nn.functional.relu(
            torch.nn.functional.max_pool2d(self.conv2_drop(self.conv2(x)), 2)
        )
        x = x.view(-1, 320)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.fc2(x)
        return torch.nn.functional.log_softmax(x, dim=1)

In [ ]:
def record_performance(model, data_loader, device, epoch, metrics, metric_type="Test"):
    """
    Record performance metric for every epoch, these metrics will be uploaded 
    after the training is finished
    """
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += torch.nn.functional.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    loss /= len(data_loader.dataset)
    accuracy = 100.0 * correct / len(data_loader.dataset)
    # record metrics
    loss_metric = {
        "MetricName": metric_type + ":loss",
        "Value": loss,
        "Step": epoch,
        "Timestamp": time.time(),
    }
    metrics.append(loss_metric)
    accuracy_metric = {
        "MetricName": metric_type + ":accuracy",
        "Value": accuracy,
        "Step": epoch,
        "Timestamp": time.time(),
    }
    metrics.append(accuracy_metric)

In [ ]:
def train_model(
    trial_component, train_set, test_set, data_dir="mnist_data", optimizer="sgd", epochs=10, hidden_channels=10
):
    """
    Function that trains the CNN classifier to identify the MNIST digits.
    Args:
        trial_component (sagemaker.core.resources.Run): SageMaker Experiment run object
        train_set (torchvision.datasets.mnist.MNIST): train dataset
        test_set (torchvision.datasets.mnist.MNIST): test dataset
        data_dir (str): local directory where the MNIST datasource is stored
        optimizer (str): the optimization algorthm to use for training your CNN
                         available options are sgd and adam
        epochs (int): number of complete pass of the training dataset through the algorithm
        hidden_channels (int): number of hidden channels in your model
    """

    # log the parameters of your model
    training_parameters = {
        "device": TrialComponentParameterValue(string_value="cpu"),
        "data_dir": TrialComponentParameterValue(string_value=data_dir),
        "optimizer": TrialComponentParameterValue(string_value=optimizer),
        "epochs": TrialComponentParameterValue(number_value=epochs),
        "hidden_channels": TrialComponentParameterValue(number_value=hidden_channels),
    }
    trial_component.update(parameters=training_parameters)

    # train the model on the CPU (no GPU)
    device = torch.device("cpu")

    # set the seed for generating random numbers
    torch.manual_seed(42)

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1000, shuffle=True)
    model = Net(hidden_channels, kernel_size=5, drop_out=0.5).to(device)
    model = torch.nn.DataParallel(model)
    momentum = 0.5
    lr = 0.01
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
    metrics = []
    for epoch in range(1, epochs + 1):
        logger.info(f"Training Epoch: {epoch}")
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader, 1):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.functional.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        record_performance(model, train_loader, device, epoch, metrics, "Train")
        record_performance(model, test_loader, device, epoch, metrics, "Test")
            
    trial_component.batch_put_metrics(MetricData=metrics)

### Start the first run in your experiment

You can use the `train_model` function with `trial_component` as parameter to start a run. Here we train the CNN with 5 hidden channels and ADAM as optimizer.

In [ ]:
train_model(
    trial_component=trial_component,
    train_set=train_set,
    test_set=test_set,
    epochs=5,
    hidden_channels=2,
    optimizer="adam",
)

In the SageMaker Experiments UI, you can observe that the new model parameters are added to the run. The model training metrics are captured and can be used to plot graphs in Experiments -> select experiment -> Runs -> Analyze.

<img src="images/experiment_run_parameters.png" width="100%" style="float: left;" />
<img src="images/experiment_run_metrics.png" width="100%" style="float: left;" />
<img src="images/experiment_run_analyze_plot.png" width="100%" style="float: left;" />

### Run multiple experiments

You can now create multiple runs of your experiment using the functions created before

In [ ]:
# define the list of parameters to train the model with
num_hidden_channel_param = [5, 10]
optimizer_param = ["adam", "sgd"]
run_id = 0
# train the model using SageMaker Experiments to track the model parameters,
# metrics and performance
for i, num_hidden_channel in enumerate(num_hidden_channel_param):
    for k, optimizer in enumerate(optimizer_param):
        run_id += 1
        run_name = "local-experiment-run-" + str(run_id) + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
        
        # Defining an experiment run for each model training run
        trial_component = TrialComponent.create(
            trial_component_name=run_name,
            parameters=trial_component_parameters,
            input_artifacts=trial_component_input_artifacts,
        )
        trial_component.associate_trail(trial_name=run_group_name)
        
        logger.info(
            f"{run_name}: Training model with {num_hidden_channel} hidden channels and {optimizer} as optimizer"
        )
        train_model(
            trial_component=trial_component,
            train_set=train_set,
            test_set=test_set,
            epochs=5,
            hidden_channels=num_hidden_channel,
            optimizer=optimizer,
        )

In [ ]:
<img src="images/experiment_runs_comparison.png" width="100%" style="float: left;" />